# HW 1. Логистическая регрессия

In [1]:
#from google.colab import drive
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

## Загрузка данных

Для того чтобы загрузить данные в нейросеть или более простые алгоритмы ML, их необходимо должным образом открыть и преобразовать в вектор с числами. Для этого воспользуемся функцией ```read_files()```.

Этот парсер делает следующие вещи:
 - открывает файл картинки с диска (с помощью библиотеки opencv),
 - проверяет, что картинка действительно открылась и сейчас является матрицей (```np.array```),
 - преобразует матрицу в вектор (путем записи всех столбцов друг под другом),
 - возвращает массив из векторов, в которых хранятся картинки, и лейбл, соответствующий каждой картинке.

In [2]:
def read_files(path: str, ans: int, target_dim: tuple = (256, 256)):
    files = os.listdir(path)
    X = None
    for i, name in enumerate(files):
        img = cv2.imread(path + '/' + name, 0) # 0 means black-white picture
        if img.shape != 0:
            img = cv2.resize(img, target_dim)
            vect = img.reshape(1, 256 ** 2) / 255.

            X = vect if (X is None) else np.vstack((X, vect))
        # print(f"{i}/{len(files)}")
    # print()
    y = np.ones((len(files))) * ans
    return X, y

Логика программы следующая:
 - есть класс ```LogisticRegression```. Он содержит веса модели и нужен для того, чтобы генерировать предсказания. Также, этот класс снабжен методом backward() для возможности дифференцирования функции потерь по весам этого класса.
 - есть класс ```Loss```. Он определяет функцию потерь, которую мы хотим использовать. Также, этот класс осуществляет подсчет градиентов функции потерь по всем весам модели.
 - есть класс ```Optimizer```. Он отвечает за то, как будут обновляться веса после подсчета градиентов (после работы класса ```Loss```). Сегодня мы рассматриваем простой градиентный спуск, но более сложные модели используют более усовершенствованные алгоритмы оптимизации.

То есть:
 1. Вызывается метод ```forward()``` нашей модели (логистической регрессии). После этого у нас выводятся текущие предсказания модели.
 2. Вызывается метод ```loss()```, который сравнивает предсказания модели с истинными ответами. После этого подсчитываются градиенты функции потерь по всем весам модели.
 3. Подсчитанные градиенты вычитаются из весов (происходит шаг градиентного спуска) путем вызова ```optimizer.step()```

## Логистическая регрессия. Backward propagation
Back propagation реализуется с помощью уравнений, написанных ниже. Для подсчитанной функции потерь:
$$
\mathcal{L} = -\frac{1}{m} \sum_{i=1}^{m} y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)
$$
где $\hat{y}_i = \sigma(w^T x_i + b)$, где $\sigma$ - сигмоидная функция.

После длительных вычислений, мы получаем, что:
$$
\frac{\partial \mathcal{L}}{\partial w} = \frac{1}{m} X^T (\hat{y} - y)
$$
$$
\frac{\partial \mathcal{L}}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i)
$$
Таким образом, мы можем обновить веса модели с помощью градиентного спуска:
$$
w = w - \alpha \frac{\partial \mathcal{L}}{\partial w} = w - \alpha \frac{1}{m} X^T (\hat{y} - y)
$$
$$
b = b - \alpha \frac{\partial \mathcal{L}}{\partial b}  = b - \alpha \frac{1}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i)
$$
где $\alpha$ - скорость обучения (learning_rate).   


In [3]:
class LogisticRegression:
    def __init__(self, n_features):
        self.w = np.zeros(n_features)
        self.b = 0.0

    def forward(self, X):
        z = np.dot(X, self.w) + self.b
        return self.sigmoid(z)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
class Loss:
    def __init__(self):
        pass

    def __call__(self, y_pred, y_true):
        eps= 1e-15  # Малое значение для предотвращения ошибок логарифма
        y_pred = np.clip(y_pred, eps, 1 - eps)  # Ограничение значений
        loss = - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return np.mean(loss)

    def backward(self, y_pred, y, X):
        dz = y_pred - y
        dw = np.dot(X.T, dz) / len(y)
        db = np.mean(dz)      
        return dw, db

class Optimizer:
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def step(self, model, dw, db):
        model.w -= self.learning_rate * dw
        model.b -= self.learning_rate * db

# Задание

1. Загрузить датасет, разбить его на два датасета: первый для обучения, второй - для проверки качества (см. функцию [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html))
2. Посчитать производные функции потерь по w и по b
3. Реализовать цикл обучения, обновляя параметры логистической регрессии:
$$w = w - \alpha \cdot \frac{\partial L}{\partial w}$$
<!-- <br> -->
$$b = b - \alpha \cdot \frac{\partial L}{\partial b}$$
Обратите внимание на величину $\alpha$. Ее надо подобрать, иначе алгоритм не будет обучаться.

4. Посчитать финальное качество модели по метрике [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)



In [4]:
path_to_box = 'lesson1_dataset/box' 
path_to_no_box = 'lesson1_dataset/no_box'

X_box, y_box = read_files(path_to_box, ans=1)
X_no_box, y_no_box = read_files(path_to_no_box, ans=0)

X = np.vstack((X_box, X_no_box))
y = np.concatenate((y_box, y_no_box), axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

input_size = X_train.shape[1]
model = LogisticRegression(input_size)
loss_function = Loss()
optimizer = Optimizer(learning_rate=0.01) #  меняем это

num_epochs = 100 # меняем это
for epoch in range(num_epochs):
    # forward propagation
    y_pred = model.forward(X_train)

    # backward propagation
    loss = loss_function(y_pred, y_train)
    dw, db = loss_function.backward(y_pred, y_train, X_train)

    # Обновление весов
    optimizer.step(model, dw, db)

    # Вывод значений потерь (каждую 10-ю эпоху)
    if epoch % 10 == 0:
        print(f"Эпоха {epoch}, loss: {loss:.4f}")

y_pred_test = model.forward(X_test)
y_pred_class = (y_pred_test > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_class)
print(f"Точность на тестовой выборке: {accuracy:.4f}")

Эпоха 0, loss: 0.6931
Эпоха 10, loss: 0.1727
Эпоха 20, loss: 0.1291
Эпоха 30, loss: 0.1042
Эпоха 40, loss: 0.0870
Эпоха 50, loss: 0.0744
Эпоха 60, loss: 0.0647
Эпоха 70, loss: 0.0571
Эпоха 80, loss: 0.0510
Эпоха 90, loss: 0.0460
Точность на тестовой выборке: 1.0000
